In [1]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from sqlite3 import Error
from pprint import pprint
import time
import pandas as pd

# https://medium.com/alpha-vantage/get-started-with-alpha-vantage-data-619a70c7f33a
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.techindicators import TechIndicators

# Import API key
from api_keys import api_key

In [2]:
def storeData(df_Stock): 
    engine = create_engine(r"sqlite:///static/data/stocks.sqlite")
    conn = engine.connect()
    
    df_Stock.reset_index(inplace=True)
    df_Stock.to_sql(name="stocks", con=conn, if_exists="replace", index=True)
    
    myDF = pd.read_sql("select * from stocks", conn)
    conn.close()
    
    print(myDF)

# end storeData()

In [3]:
def refresh():
    old_to_new = {"1. open": "open", "2. high": "high", "3. low": "low", "4. close": "close", "5. volume": "volume"}
    reorder = ["ticker", "date", "open", "high", "low", "close", "volume"]
    # Create a list of ticker symbols
    stocks = ["DAL", "AAL", "LUV", "UAL", "MSFT", "AAPL", "V"] #, "INTC", "UBER", "BKNG", "LYFT", "TCOM", "ZM"]
    
    for stockTicker in stocks:
        print(f"Getting data for ticker:'{stockTicker}'")
        
        try:
            # Chose your output format, or default to JSON (python dict)
            ts = TimeSeries(api_key, output_format='pandas')
            
            # aapl_data is a pandas dataframe, aapl_meta_data is a dict
            stockData, meta_data = ts.get_daily(symbol=stockTicker)
            stockData["ticker"] = stockTicker
            stockData.reset_index(inplace=True)
            stockData.rename(columns=old_to_new, inplace=True)
            stockData = stockData[reorder]
            
            stockData[["open", "high", "low", "close", "volume"]] = stockData[["open", "high", "low", "close", "volume"]].apply(pd.to_numeric)
            
            storeData(stockData)
            
            pprint(stockData)
            break
            time.sleep(2)
        except Error as e:
            print(f"The API failed for the stock '{stockTicker}'. Please try again later./n/n  Error is:  {e}")
        finally:
            print("The update process has completed.")
        # End Try
    # Next stockTicker
# End refresh()

In [4]:
refresh()

Getting data for ticker:'DAL'
    level_0  index ticker                        date     open     high  \
0         0      0    DAL  2020-04-15 00:00:00.000000  24.9100  25.5000   
1         1      1    DAL  2020-04-14 00:00:00.000000  23.9899  25.2900   
2         2      2    DAL  2020-04-13 00:00:00.000000  24.9800  25.0600   
3         3      3    DAL  2020-04-09 00:00:00.000000  25.4000  25.5300   
4         4      4    DAL  2020-04-08 00:00:00.000000  23.0900  23.6400   
..      ...    ...    ...                         ...      ...      ...   
95       95     95    DAL  2019-11-26 00:00:00.000000  56.3200  57.4800   
96       96     96    DAL  2019-11-25 00:00:00.000000  56.3900  56.6900   
97       97     97    DAL  2019-11-22 00:00:00.000000  55.5700  56.1300   
98       98     98    DAL  2019-11-21 00:00:00.000000  55.7400  55.9856   
99       99     99    DAL  2019-11-20 00:00:00.000000  56.8000  56.8500   

        low   close      volume  
0   23.3100  24.245  82226722.0  
1

In [ ]:
engine = create_engine(r"sqlite:///static/data/stocks.sqlite")
conn = engine.connect()    
myDF = pd.read_sql("select * from stocks", conn)
conn.close()

print(myDF)